# Лабораторная работа 4.  Алгоритмы на графах

Вариант - 3

## Задание - 1
Найти кратчайший путь на графе между парами вершин методом динамического программирования вручную.

| Вариант | Начальная и конечная вершины| Граф |
|:---|:---|:---|
| 3  | 2, 6 | ![](./img/003.png) |

Пусть 5 -> 4 = 1; 5 -> 6 = 1

В начале посмотрим все возможные пути из начальной вершины в конечную. 

В нашем случае начальная вершина - 2, а конечная - 6.

Из вершины 2 в вершину 6, есть только один путь, и, следовательно, кратчайшим. Он будет выглядеть следующим образом: из начальной вершины идём в вершину 5, затем приходим в конечную.

Кратчайшая путь будет равен:
- 2 -> 5 = 5 
- 5 -> 6 = 1 
- и следовательно 2 -> 6 = 5 + 1 = 6

Начальные условия $f_2=0$, $S_{22} = 0$.

Находим последовательно значения функции $f_i$ (в условных единицах) для каждой вершины ориентированного графа:

$f_5 = \min(S_{52} + f_2) = 5 + f_2 = 5 + 0 = 5$;

$f_6 = \min(S_{65} + f_5) = 5 + f_5 = 5 + 1 = 6$;

Длина кратчайшего пути составляет 6 условных единиц. 

Для выбора оптимальной траектории движения следует осуществить просмотр функций $f_i$ в обратном порядке, то есть с $f_{6}$. 

Пусть $f_i = f_{6}$. 

В данном случае

$f_6 = \min(1 + f_5) = 1 + 5 = 1 + 5 = 6$;

Получаем, что $1 + f_5 = 6$, то есть $f_j = f_5$. 
Значит, из вершины 6 следует перейти к вершине 5.

Имеем $f_i = f_5$.   

Рассмотрим функцию 
$f_5 = \min(5 + f_2) = 5 + f_2 = 5 + 0 = 5$;

т.е. $f_j = f_5$ и т.д.

Таким образом, получаем кратчайший путь от вершины 2 к вершине 6: (2,  5,  6) 
	Рассмотренный метод определения кратчайшего пути может быть распространен и на неориентированные графы. 
    

## Задание - 2
Реализовать прогрммно поиск кратчайшего пути на графе между парами вершин из задания 1 методом динамического программирования.

In [44]:
def floyd_warshall(graph):
    # Копируем граф, чтобы не изменять исходный
    dist = [row[:] for row in graph]

    # Проходим по всем вершинам
    for k in range(len(dist)):
        for i in range(len(dist)):
            for j in range(len(dist)):
                # Если есть более короткий путь от i к j через k
                if dist[i][j] > dist[i][k] + dist[k][j]:
                    # Обновляем расстояние
                    dist[i][j] = dist[i][k] + dist[k][j]
    
    return dist

graph = [[0, 8, 3, float('inf'), float('inf'), float('inf'), float('inf')],
         [float('inf'), 0, float('inf'), 1, 5, float('inf'), float('inf')],
         [float('inf'), float('inf'), 0, float('inf'), 4, 12, float('inf')],
         [float('inf'), float('inf'), float('inf'), 0, float('inf'), float('inf'), 6],
         [float('inf'), float('inf'), float('inf'), 1, 0, 1, 12], # Заменить 5 -> 4 и 5 -> 6 
         [float('inf'), float('inf'), float('inf'), float('inf'), float('inf'), 0, 7],
         [float('inf'), float('inf'), float('inf'), float('inf'), float('inf'), float('inf'), 0]]

# Находим кратчайшие расстояния между всеми парами вершин
dist = floyd_warshall(graph)

# Выводим результат
for row in dist:
    print(row)

[0, 8, 3, 8, 7, 8, 14]
[inf, 0, inf, 1, 5, 6, 7]
[inf, inf, 0, 5, 4, 5, 11]
[inf, inf, inf, 0, inf, inf, 6]
[inf, inf, inf, 1, 0, 1, 7]
[inf, inf, inf, inf, inf, 0, 7]
[inf, inf, inf, inf, inf, inf, 0]


## Задание - 3
Реализовать алгоритм Дейкстры поиска кратчайшего пути на графе между парами вершин:

| Вариант | Начальная и конечная вершины| Граф | 
|:---|:---|:---|
| 3  | 1, 5 | ![](./img/013.png) |

In [45]:
import heapq

def dijkstra(graph, start):
    """
    Алгоритм Дейкстры поиска кратчайшего пути на графе между парами вершин.

    Аргументы:
    - graph: словарь, представляющий граф в виде списков смежности
    - start: начальная вершина

    Возвращает:
    - distances: словарь, содержащий расстояния от начальной вершины до всех остальных вершин графа
    - previous: словарь, содержащий предыдущие вершины на кратчайших путях до всех остальных вершин графа
    """

    # Инициализация расстояний до всех вершин графа
    distances = {vertex: float('inf') for vertex in graph}
    distances[start] = 0

    # Создание очереди с приоритетами (heap), которая будет хранить вершины графа в порядке возрастания расстояний до них
    priority_queue = [(0, start)]

    # Инициализация словаря, который будет хранить предыдущие вершины на кратчайших путях до всех остальных вершин графа
    previous = {vertex: None for vertex in graph}

    while priority_queue:
        # Извлечение вершины из очереди с приоритетами, которая имеет минимальное расстояние до начальной вершины
        current_distance, current_vertex = heapq.heappop(priority_queue)

        # Если расстояние до текущей вершины уже больше, чем минимальное расстояние из начальной вершины до неё, то пропустить её
        if current_distance > distances[current_vertex]:
            continue

        # Проход по всем соседним вершинам текущей вершины
        for neighbor, weight in graph[current_vertex].items():
            # Расчёт расстояния от начальной вершины до текущей соседней вершины через текущую вершину
            distance = current_distance + weight

            # Если новое расстояние меньше, чем ранее известное расстояние до этой соседней вершины, то обновить расстояние и предыдущую вершину на пути до неё
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                previous[neighbor] = current_vertex

                # Добавить соседнюю вершину в очередь с приоритетами с новым расстоянием до неё
                heapq.heappush(priority_queue, (distance, neighbor))

    return distances

graph = {
    '1': {'2': 6, '5': 2},
    '2': {'1': 6, '3': 9, '5': 1, '6': 5, '7': 3},
    '3': {'2': 9, '4': 12, '7': 8},
    '4': {'3': 12, '7': 3, '8': 4},
    '5': {'1': 2, '2': 1, '6': 7},
    '6': {'2': 5, '5': 7, '7': 4},
    '7': {'2': 3, '3': 8, '4': 3, '6': 4, '8': 16},
    '8': {'4': 4, '7': 16}
}

dijkstra(graph, '1')

{'1': 0, '2': 3, '3': 12, '4': 9, '5': 2, '6': 8, '7': 6, '8': 13}

## Задание - 4
Реализовать прогрммно один из алгоритмов поиска кратчайшего пути на графе между парами вершин из задания 3.

In [46]:
def floyd_warshall(graph):
    # Копируем граф, чтобы не изменять исходный
    dist = [row[:] for row in graph]

    # Проходим по всем вершинам
    for k in range(len(dist)):
        for i in range(len(dist)):
            for j in range(len(dist)):
                # Если есть более короткий путь от i к j через k
                if dist[i][j] > dist[i][k] + dist[k][j]:
                    # Обновляем расстояние
                    dist[i][j] = dist[i][k] + dist[k][j]
    
    return dist

graph = [[0, 6, float('inf'), float('inf'), 2, float('inf'), float('inf'), float('inf')],
         [6, 0, 9, float('inf'), 1, 5, 3, float('inf')],
         [float('inf'), 9, 0, 12, float('inf'), float('inf'), 8, float('inf')],
         [float('inf'), float('inf'), 12, 0, float('inf'), float('inf'), 3, 4],
         [2, 1, float('inf'), float('inf'), 0, 7, float('inf'), float('inf')],
         [float('inf'), 5, float('inf'), float('inf'), 7, 0, 4, float('inf')],
         [float('inf'), 3, 8, 3, float('inf'), 4, 0, 16],
         [float('inf'), float('inf'), float('inf'), 4, float('inf'), float('inf'), 16, 0]]

# Находим кратчайшие расстояния между всеми парами вершин
dist = floyd_warshall(graph)

# Выводим результат
for row in dist:
    print(row)

[0, 3, 12, 9, 2, 8, 6, 13]
[3, 0, 9, 6, 1, 5, 3, 10]
[12, 9, 0, 11, 10, 12, 8, 15]
[9, 6, 11, 0, 7, 7, 3, 4]
[2, 1, 10, 7, 0, 6, 4, 11]
[8, 5, 12, 7, 6, 0, 4, 11]
[6, 3, 8, 3, 4, 4, 0, 7]
[13, 10, 15, 4, 11, 11, 7, 0]
